# Módulos add and norm y feed forward finales

Solo nos quedan los módulos `Add & Norm` y `Feed Forwar`

<div style="text-align:center;">
  <img src="Imagenes/transformer_architecture_model_add_and_norm4.png" alt="Add and norm" style="width:425px;height:626px;">
  <img src="Imagenes/transformer_architecture_model_decoder_feed_forward.png" alt="Encoder Feed Forward" style="width:425px;height:626px;">
  <img src="Imagenes/transformer_architecture_model_add_and_norm5.png" alt="Add and norm" style="width:425px;height:626px;">
</div>

Como ya los hemos explicado, estos tres últimos los volvemos a contar en un único notebook

## Add & Norm

Estos módulos aportan beneficios a la red por las conexiones residuales

## Feed Forward

Consiste en una red fully conected como las que hemos visto al principio del curso, pero de dos capas, exactamente en el paper lo definen así

<div style="text-align:center;">
  <img src="Imagenes/feed_forward_equation.png" alt="Feed Forward equation">
</div>

Esta fórmula lo que nos indica es que hay que pasar la matriz por una capa lineal con pesos $W_1$ y $b_1$, aplicarle una función de activación Relu y a continuación pasarle por otra capa lineal con pesos $W_2$ y $b_2$

El motivo de esta capa es añadir una no linealidad al encoder, lo cual trae las siguientes ventajas

 * Incremento de la capacidad del modelo:
 * Extracción de características:

En la primera capa del `Feed Forward` se podrá aumentar la dimensión de la matriz, pero a la salida del módulo `Feed Forward` la dimensión de la matriz tiene que ser $\left(m_E \times n_E\right)$.

En el paper proponen que si la matriz de entrada tiene una dimensión de embedding de 512, a la salida de la primera capa la matriz tenga una dimensión de embedding de 2048 y a la salida de la segunda capa vuelva a tener una dimensión de embedding de 512. Por lo que en la parte de `Feed Forward` internamente se multiplica la dimensión de la matriz por 4 y luego se vuelve a reducir

## Implementación

Recuperamos las clases que ya hemos hecho antes

In [1]:
import torch
import torch.nn as nn

class AddAndNorm(nn.Module):
    def __init__(self, dim_embedding):
        """
        Args:
            dim_embedding (int): Embedding dimension.
        """
        super().__init__()
        self.normalization = nn.LayerNorm(dim_embedding)

    def forward(self, x, sublayer):
        """
        Args:
            x (torch.Tensor): Input tensor.
            sublayer (torch.Tensor): Sublayer tensor.

        Returns:
            torch.Tensor: Output tensor.
        """
        return self.normalization(torch.add(x, sublayer))

In [2]:
import torch.nn.functional as F

class FeedForward(nn.Module):
    def __init__(self, dim_embedding, increment=4):
        super().__init__()
        self.feed_forward = nn.Sequential(
            nn.Linear(dim_embedding, dim_embedding*increment),
            nn.ReLU(),
            nn.Linear(dim_embedding*increment, dim_embedding)
        )
    
    def forward(self, x):
        """
        Args:
            x (torch.Tensor): (batch_size, seq_len, dim_embedding)

        Returns:
            torch.Tensor: (batch_size, seq_len, dim_embedding)
        """
        x = self.feed_forward(x)
        return x